In [7]:
!pip install keras.preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
!pip install pad_sequences
import pad_sequences

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import tensorflow as tf 

from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# read data
reviews_df = pd.read_csv("/content/drive/MyDrive/Hotel_ReviewsNEW.csv")
# append the positive and negative text reviews
reviews_df["review"] = reviews_df["Negative_Review"] + reviews_df["Positive_Review"]
# create the label
reviews_df["is_bad_review"] = reviews_df["Reviewer_Score"].apply(lambda x: 1 if x < 5 else 0)
# select only relevant columns
reviews_df = reviews_df[["review", "is_bad_review"]]
reviews_df.head()


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,2,4,5,6,9,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,review,is_bad_review
0,No Negative No real complaints the hotel was g...,0
1,No Negative This hotel is being renovated with...,0
2,No Negative This hotel is awesome I took it si...,0
3,No Negative Public areas are lovely and the ro...,0
4,No Negative The quality of the hotel was brill...,0


In [ ]:
reviews_df

,review,is_bad_review
0,No Negative No real complaints the hotel was g...,0
1,No Negative This hotel is being renovated with...,0
2,No Negative This hotel is awesome I took it si...,0
3,No Negative Public areas are lovely and the ro...,0
4,No Negative The quality of the hotel was brill...,0
...,...,...
515729,NaN,0
515730,NaN,0
515731,NaN,0
515732,NaN,0


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
X=reviews_df['review']
Y= reviews_df.is_bad_review
X,Y

(0         No Negative No real complaints the hotel was g...
 1         No Negative This hotel is being renovated with...
 2         No Negative This hotel is awesome I took it si...
 3         No Negative Public areas are lovely and the ro...
 4         No Negative The quality of the hotel was brill...
                                 ...                        
 515729                                                  NaN
 515730                                                  NaN
 515731                                                  NaN
 515732                                                  NaN
 515733                                                  NaN
 Name: review, Length: 515734, dtype: object, 0         0
 1         0
 2         0
 3         0
 4         0
          ..
 515729    0
 515730    0
 515731    0
 515732    0
 515733    0
 Name: is_bad_review, Length: 515734, dtype: int64)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
Y=le.fit_transform(Y)
Y

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
Y=Y.reshape(-1,1)
Y

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [ ]:
from nltk.corpus import stopwords   #remove stopwords
from nltk.stem.porter import PorterStemmer   #stemming
import re
#Different models for converting text to vector
from sklearn.feature_extraction.text import CountVectorizer #BOW
from gensim.models import Word2Vec

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
clean_review=reviews_df.copy()

In [ ]:
import string
def review_cleaning(review):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    review = str(review).lower()
    review = re.sub('\[.*?\]', '', review)
    review = re.sub('https?://\S+|www\.\S+', '', review)
    review = re.sub('<.*?>+', '', review)
    review= re.sub('[%s]' % re.escape(string.punctuation), '', review)
    review = re.sub('\n', '', review)
    review = re.sub('\w*\d\w*', '', review)
    return review
clean_review['review']=clean_review['review'].apply(lambda x:review_cleaning(x))
clean_review.head()

,review,is_bad_review
0,no negative no real complaints the hotel was g...,0
1,no negative this hotel is being renovated with...,0
2,no negative this hotel is awesome i took it si...,0
3,no negative public areas are lovely and the ro...,0
4,no negative the quality of the hotel was brill...,0


In [ ]:
stop = stopwords.words('english')
clean_review['review'] = clean_review['review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
clean_review.head()

,review,is_bad_review
0,negative real complaints hotel great great loc...,0
1,negative hotel renovated great care appreciati...,0
2,negative hotel awesome took sincirely bit chea...,0
3,negative public areas lovely room nice window ...,0
4,negative quality hotel brilliant even though r...,0


In [ ]:
review_features=clean_review.copy()
review_features=review_features[['review']].reset_index(drop=True)
review_features.head()

,review
0,negative real complaints hotel great great loc...
1,negative hotel renovated great care appreciati...
2,negative hotel awesome took sincirely bit chea...
3,negative public areas lovely room nice window ...
4,negative quality hotel brilliant even though r...


In [ ]:
stop_words = set(stopwords.words("english"))
#Performing stemming on the review dataframe
ps = PorterStemmer()

#splitting and adding the stemmed words except stopwords
corpus = []
for i in range(0, len(review_features)):
    trip = re.sub('[^a-zA-Z]', ' ', review_features['review'][i])
    trip= trip.lower()
    trip = trip.split()
    trip = [ps.stem(word) for word in trip if not word in stop_words]
    trip = ' '.join(trip)
    corpus.append(trip)#Getting the target variable
y=clean_review['review']

In [ ]:
len(corpus),len(y)
print(Y)

[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]


In [ ]:
from sklearn.model_selection import train_test_split
xtrain,xval,ytrain,yval=train_test_split(corpus,Y,test_size=0.2,random_state=2)

In [ ]:
len(xtrain),len(ytrain),len(xval),len(yval)

(412587, 412587, 103147, 103147)

In [ ]:
docs=[text.split() for text in xtrain]

In [ ]:
len(docs)

412587

In [ ]:
import gensim
w2v_model = gensim.models.Word2Vec(size=300, window=3,min_count=5, workers=8)

In [ ]:
w2v_model.build_vocab(docs)

In [ ]:
words = w2v_model.wv.vocab.keys()
vocab_size = len(words)
print("Vocab size", vocab_size)

Vocab size 9197


In [ ]:
w2v_model.train(docs,total_examples=len(docs),epochs=32)

(92364351, 128296928)

In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer()
tokenizer.fit_on_texts(xtrain)
vocab_size = len(tokenizer.word_index) + 1
print("Total words", vocab_size)

from keras_preprocessing.sequence import pad_sequences
x_train = pad_sequences(tokenizer.texts_to_sequences(xtrain), maxlen=300)
x_test = pad_sequences(tokenizer.texts_to_sequences(xval), maxlen=300)

Total words 36687


In [ ]:
x_train

array([[  0,   0,   0, ...,   4,  60,  34],
       [  0,   0,   0, ...,   7, 186,   9],
       [  0,   0,   0, ...,  30,  77,  29],
       ...,
       [  0,   0,   0, ...,  11,  10,   2],
       [  0,   0,   0, ...,  32,  89,  80],
       [  0,   0,   0, ..., 592,  60, 145]], dtype=int32)

In [ ]:
import numpy as np
len(x_train),len(ytrain),len(x_test),len(yval)
embedding_matrix = np.zeros((vocab_size, 300))
print(embedding_matrix)
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]
print(embedding_matrix.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(36687, 300)


In [ ]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.08487152,  0.39116743,  0.48666647, ..., -0.40508556,
         0.43122885, -0.45017269],
       [-1.5461911 , -0.17187105,  0.81950009, ..., -0.44177303,
        -0.43406636,  0.23061505],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [ ]:
from keras.layers import Activation, Dense, Dropout, Embedding
embedding_layer = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=300, trainable=False)

In [ ]:
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM
# Build The model
model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.summary()

model.compile(loss='binary_crossentropy',optimizer="adam",metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 300)          11006100  
                                                                 
 dropout (Dropout)           (None, 300, 300)          0         
                                                                 
 lstm (LSTM)                 (None, 64)                93440     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 11,099,605
Trainable params: 93,505
Non-trainable params: 11,006,100
_________________________________________________________________


In [ ]:
print(x_train)

[[  0   0   0 ...   4  60  34]
 [  0   0   0 ...   7 186   9]
 [  0   0   0 ...  30  77  29]
 ...
 [  0   0   0 ...  11  10   2]
 [  0   0   0 ...  32  89  80]
 [  0   0   0 ... 592  60 145]]


In [ ]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
callbacks = [ ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),EarlyStopping(monitor='val_acc', min_delta=1e-4, patience=5)]

history = model.fit(x_train, ytrain,batch_size=32,epochs=5,validation_split=0.1,verbose=1, callbacks=callbacks)

Epoch 1/5
10681/11604 [==========================>...] - ETA: 5:51 - loss: 0.0595 - accuracy: 0.9825

In [ ]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
callbacks = [ ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),EarlyStopping(monitor='val_acc', min_delta=1e-4, patience=5)]

#history = model.fit(x_train, ytrain,batch_size=32,epochs=10,validation_split=0.1,verbose=1,callbacks=callbacks)

In [ ]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
callbacks = [ ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),EarlyStopping(monitor='val_acc', min_delta=1e-4, patience=5)]

#history = model.fit(x_train, ytrain,batch_size=32,epochs=15,validation_split=0.1,verbose=1,callbacks=callbacks)

In [ ]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
callbacks = [ ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),EarlyStopping(monitor='val_acc', min_delta=1e-4, patience=5)]

#history = model.fit(x_train, ytrain,batch_size=32,epochs=20,validation_split=0.1,verbose=1,callbacks=callbacks)

In [ ]:
score = model.evaluate(x_test, yval, batch_size=32)
print()
print("ACCURACY:",score[1])
print("LOSS:",score[0])

In [ ]:
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
y_pred=model.predict(x_test)
y_pred=np.argmax(y_pred,axis=1)
#Creating confusion matrix
confusion_matrix(yval,y_pred)